In [1]:
from glob import glob 
from fastcore.helpers import load_pickle

In [2]:
all_res = glob('results/20221117_solubility/*')

In [3]:
len(all_res)

3

In [5]:
load_pickle(all_res[0])

{'num_train_points': 500,
 'baseline_cm': pycm.ConfusionMatrix(classes: ['large', 'medium', 'small', 'very large', 'very small']),
 'baseline_y_eval': array(['small', 'very large', 'medium', 'medium', 'large', 'medium',
        'very large', 'medium', 'medium', 'medium', 'very small', 'small',
        'very large', 'very large', 'large', 'small', 'very large',
        'small', 'very small', 'large', 'very large', 'medium',
        'very small', 'medium', 'small', 'large', 'medium', 'medium',
        'very large', 'large', 'medium', 'very small', 'very small',
        'medium', 'small', 'medium', 'medium', 'medium', 'medium', 'small',
        'large', 'medium', 'very small', 'very small', 'medium', 'medium',
        'medium', 'large', 'very large', 'large', 'medium', 'very small',
        'very large', 'small', 'large', 'large', 'very small', 'small',
        'very small', 'small', 'medium', 'small', 'medium', 'very large',
        'small', 'medium', 'very small', 'very large', 'large',

In [4]:
all_res

['results/20221117_solubility/results_solubility_500.pkl',
 'results/20221117_solubility/results_solubility_50.pkl',
 'results/20221117_solubility/results_solubility_10.pkl']